# Cheat sheet for EDA

**Environment preparation**

* Create Python environment
```
python -m venv env-name
```
* Run venv
```
env-name\Scripts\Activate
```
* Intall python packages
```
pip install jupyter jupyterlab pandas matplotlib requests seaborn scipy scikit-learn
```
* Run Jupyter lab
```
jupyter lab
```

## Important links

**Pandas documentation:** [https://pandas.pydata.org/docs/reference/index.html](https://pandas.pydata.org/docs/reference/index.html)

**Seaborn documentation:** [https://seaborn.pydata.org/api.html](https://seaborn.pydata.org/api.html)

## Import used packages

In [ ]:
import pandas as pd # dataframes
import numpy as np # matrices and linear algebra
import matplotlib.pyplot as plt # plotting
import seaborn as sns # another matplotlib interface - styled and easier to use

# Pandas

## Read the data

In [ ]:
df_full = pd.read_csv('https://raw.githubusercontent.com/lubsar/EFREI-Introduction-to-Machine-Learning/main/datasets/zsu_cv1_data.csv', sep=',')

## Show missing values counts by columns

### Show 5 first and last rows

In [ ]:
df_full

### Show first 5 rows

In [ ]:
df_full.head()

### Show last 20 rows

In [ ]:
df_full.tail(20)

In [ ]:
df_full.isna().sum().sort_values(ascending=False).head(20)

## Get summary of the data

In [ ]:
df_full.describe()

#### Row and column count

In [ ]:
df_full.shape

#### Datatypes of columns

In [ ]:
df_full.dtypes

#### Column names

In [ ]:
df_full.columns

#### Row index values

In [ ]:
df_full.index

## Select column to Pandas Series

In [ ]:
price = df_full['SalePrice'] # df_full.SalePrice
price

## Data filtering using .loc and .iloc

### The .iloc[] indexer works with positional indexes - very close to the way of working with the raw arrays
### The .loc[] indexer works with column names and logical expressions

### Select all rows and 3rd column of dataframe

In [ ]:
df_full.iloc[:, 2]

### Select all rows and LAST column of dataframe

In [ ]:
df_full.iloc[:, -1]

### Select rows 15 to 22 and all columns

In [ ]:
df_full.iloc[15:23, :]

### Select rows 15 to 22 and 3rd column

In [ ]:
df_full.iloc[15:23, 3]

## Select only a subset of columns to a new dataframe

In [ ]:
df = df_full.loc[:, ['Id', 'SalePrice', 'GarageFinish', 'MSSubClass','BldgType','HouseStyle','OverallQual','OverallCond','YearBuilt','Heating','CentralAir','GrLivArea','BedroomAbvGr']].copy()
df.head()

### Select houses cheaper than 180k USD and with at least 2 bedrooms

In [ ]:
df.loc[(df.SalePrice < 180000) & (df.BedroomAbvGr >= 2), :]

### Add a new column named Age for each house (current year - year built)

In [ ]:
df.loc[:, 'Age'] = 2021 - df.YearBuilt

### Add a new column IsLuxury with True value for houses with more than 3 bedrooms and price above 214k USD (.loc)
- How many luxury houses are in the dataset?

In [ ]:
df['IsLuxury'] = False
df.loc[(df.SalePrice > 214000) & (df.BedroomAbvGr > 3), 'IsLuxury'] = True

### Compute counts for all the heating methods (groupby / value_counts)

In [ ]:
df.groupby('Heating').Heating.count()

In [ ]:
df.Heating.value_counts()

# Visualization

## Scatter plot

In [ ]:
fig = plt.figure(figsize=((9,6)))
sns.scatterplot(data=df, x='GrLivArea', y='SalePrice', hue='CentralAir', palette='deep')

## Line plot

In [ ]:
avg_prices = df.groupby('YearBuilt').SalePrice.mean().reset_index(name='AvgSalePrice')
plt.figure(figsize=(9,6))
sns.lineplot(data=avg_prices, x='YearBuilt', y='AvgSalePrice')

## Bar plot

In [ ]:
df.loc[df.YearBuilt < 2000, 'AgeCat'] = '<2000'
df.loc[df.YearBuilt >= 2000, 'AgeCat'] = '>=2000'
df_number_of_categories = df.groupby(['BldgType', 'AgeCat']).AgeCat.count().reset_index(name='Count')
df_number_of_categories

In [ ]:
sns.barplot(data=df_number_of_categories, y='BldgType', x='Count', hue='AgeCat')

## Histogram

In [ ]:
fig = plt.figure(figsize=(9, 6))
sns.histplot(df.SalePrice, bins=60, edgecolor='white', color='green')
plt.axvline(df.SalePrice.quantile(0.25), color='red')
plt.axvline(df.SalePrice.quantile(0.75), color='red')

In [ ]:
sns.displot(data=df, x='SalePrice', hue='GarageFinish', edgecolor='white')

## Boxplot

In [ ]:
fig = plt.figure(figsize=(16, 9))
sns.boxplot(data=df, y='SalePrice', x='BldgType')

## Pair plot

In [ ]:
selected_columns = ['LotFrontage', 'LotArea', 'YearBuilt', 'GrLivArea', 'GarageYrBlt', 'GarageArea', 'SalePrice']
sns.pairplot(data=df_full, vars=selected_columns, hue='BldgType')

## Subplots

In [ ]:
qualitative_columns = df.select_dtypes(exclude=np.number).columns

fig, axes = plt.subplots(len(qualitative_columns), 2, figsize=(10, 20))

for i, col in enumerate(qualitative_columns):
    ax_left = fig.axes[2*i]
    ax_right = fig.axes[2*i + 1]
    sns.boxplot(data=df, x=col, y='SalePrice', ax=ax_left)
    ax_left.set_xticklabels(ax_left.xaxis.get_majorticklabels(), rotation=60)
    sns.countplot(data=df, x=col, ax=ax_right)
    ax_right.set_xticklabels(ax_right.xaxis.get_majorticklabels(), rotation=60)
        
fig.tight_layout()

## Heatmap and correlation

In [ ]:
df_corr = df.corr(numeric_only=True)
df_corr

In [ ]:
plt.figure(figsize=(12, 9))
sns.heatmap(df_corr, square=True, cmap='RdYlGn', vmin=-1, vmax=1, annot=True)

## Regression line in scatter plot

In [ ]:
fig = plt.figure(figsize=(12, 12))
sns.regplot(data = df, x='GrLivArea', y='SalePrice')
plt.show()